In [1]:
#!pip3 install twilio

In [2]:
import os
from twilio.rest import Client
import time
from dotenv import load_dotenv

import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

load_dotenv()

True

# Armado de la URL

In [3]:
query = 'Medellín'
api_key = os.getenv('API_KEY_WAPI')

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=f4231736355b4bb7b4821948232209&q=Medellín&days=1&aqi=no&alerts=no'

In [4]:
response = requests.get(url_clima).json()

In [5]:
response

{'location': {'name': 'Medellín',
  'region': 'Antioquia',
  'country': 'Kolumbien',
  'lat': 6.29,
  'lon': -75.54,
  'tz_id': 'America/Bogota',
  'localtime_epoch': 1695406721,
  'localtime': '2023-09-22 13:18'},
 'current': {'last_updated_epoch': 1695406500,
  'last_updated': '2023-09-22 13:15',
  'temp_c': 32.0,
  'temp_f': 89.6,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 11.9,
  'wind_kph': 19.1,
  'wind_degree': 40,
  'wind_dir': 'NE',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.19,
  'precip_in': 0.01,
  'humidity': 32,
  'cloud': 50,
  'feelslike_c': 33.8,
  'feelslike_f': 92.9,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 5.0,
  'gust_mph': 0.3,
  'gust_kph': 0.4},
 'forecast': {'forecastday': [{'date': '2023-09-22',
    'date_epoch': 1695340800,
    'day': {'maxtemp_c': 25.7,
     'maxtemp_f': 78.2,
     'mintemp_c': 12.1,
     'mintemp_f': 53.8,
  

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [8]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [9]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [10]:
response['forecast']['forecastday'][0]['hour'][0]['time']

'2023-09-22 00:00'

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0] # fecha

'2023-09-22'

In [12]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) # hora

1

In [13]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text'] # condición

'Mist'

In [14]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] # temperatura

13.0

In [15]:
response['forecast']['forecastday'][0]['hour'][2]['will_it_rain']

0

In [16]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [17]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [18]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<00:00, 24267.91it/s]


In [19]:
datos[0]

('2023-09-22', 0, 'Mist', 13.0, 0, 0)

In [20]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-09-22,0,Mist,13.0,0,0
1,2023-09-22,1,Partly cloudy,12.7,0,0
2,2023-09-22,2,Partly cloudy,12.5,0,0
3,2023-09-22,3,Partly cloudy,12.2,0,0
4,2023-09-22,4,Partly cloudy,12.2,0,0
5,2023-09-22,5,Partly cloudy,12.0,0,0
6,2023-09-22,6,Mist,12.0,0,0
7,2023-09-22,7,Partly cloudy,15.1,0,0
8,2023-09-22,8,Sunny,18.8,0,0
9,2023-09-22,9,Partly cloudy,21.2,0,0


In [31]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [21]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
11,2023-09-22,11,Patchy rain possible,24.9,1,75
17,2023-09-22,17,Patchy rain possible,17.7,1,73
18,2023-09-22,18,Light rain shower,16.1,1,71
19,2023-09-22,19,Patchy light rain,14.6,1,90
20,2023-09-22,20,Light rain shower,14.3,1,93
21,2023-09-22,21,Mist,14.2,1,87


In [22]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [23]:
df['Fecha'][0]

'2023-09-22'

In [24]:
df_rain

,Condicion
Hora,
11,Patchy rain possible
17,Patchy rain possible
18,Light rain shower
19,Patchy light rain
20,Light rain shower
21,Mist


In [25]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-09-22 en Medellín es : \n\n\n                  Condicion\nHora                      \n11    Patchy rain possible\n17    Patchy rain possible\n18       Light rain shower\n19       Patchy light rain\n20       Light rain shower\n21                    Mist'

In [27]:
os.getenv('PHONE_NUMBER')


'+14056895542'

# Mensaje SMS desde Twilio

In [29]:
time.sleep(2)
account_sid = os.getenv('TWILIO_ACCOUNT_SID') 
auth_token = os.getenv('TWILIO_AUTH_TOKEN')

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=os.getenv('PHONE_NUMBER'),
                     to=os.getenv('MIO')
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM6d671068f99eac0cdf711f85fa155831
